<a href="https://colab.research.google.com/github/spaceofsilver/crawling/blob/main/2_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91_level3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 조건

- 수집하고자 하는 데이터가 웹사이트에 게시되어 있다
- openAPI가 없다
- 사이트에 접속만 하면 그냥 볼 수 있다
  - 더보기, 로그인, 스크롤 등의 절차가 없다

## 사용되는 패키지

- BeautifulSoup/bs4

## 대상 사이트

- https://finance.naver.com/marketindex/?tabSel=exchange#tab_section

# 간단한 사이트에서 수집

In [ ]:
# 1.모듈 가져오기

from bs4 import BeautifulSoup # 파싱할때 사용
from urllib.request import urlopen

In [ ]:
# 2. 타겟사이트 접속

target_site   =  'https://finance.naver.com/marketindex/exchangeList.nhn'
res           =   urlopen( target_site )  # res찍어도 객체라는 사실만 확인
#  중요! 오픈하는순간 데이터 수집은 끝.
#  즉 웹 스크래핑은 이걸로 끝
#  밑에서부터는 파싱을 해서 돔트리를 띄운 후 객체 사이를 탐색하며 필요한 정보 수집.

In [ ]:
if res.getcode() == 200:
  print('정상적으로 잘 받았다')
else:
  print('사이트에 뭔가 문제가 있다 점검 필요')

정상적으로 잘 받았다


In [ ]:
# 3. 응답받은 데이터                  -> **HTML 데이터**( 반정형 데이터 )
#   html 데이터를 파싱하는 모듈 필요  -> 원하는 데이터 추출 : bs4로 해결 가능
#   파싱 수행을 하는 s/w              -> parser라고 한다.
#   soup = BeautifulSoup( res, 파서의 종류)
# BeautifulSoup은 ML로 대변되는 데이터 포멧을 파싱하는 대표 모듈
#   Dom 트리를 구성했다.
soup    = BeautifulSoup( res, "html5lib" )
soup

In [ ]:
# 통화명 추출 : 1) 특정 2) 추출
# css.selecctor: .tit
# body > div > table > tbody > tr:nth-child(1) > td.tit > a -> td.tit > a 이렇게만으로도 특정 가능

In [ ]:
tmp = soup.select('.tit')
# soup가 파싱을 한 결과는 DOM tree가 메모리에 올라가게 되고 
# 엘리먼트(태그) <td></td> 이런 것들이 전부 다 객체로 표현되기 때문에
# 그 이하를 내려갈 때는 .을 통해서 접근한다.
type(tmp) , len(tmp)

(list, 44)

In [ ]:
tmp[0].a.string.strip()
# select()      : 리스트로 추출
# select_one()  : 객체로 추출

'미국 USD'

In [ ]:
for td in soup.select('.tit'):
  print(td.a.string.strip())

In [ ]:
# 각 국가별 매매기준율을 추출
for td in soup.select('.sale'):
  print(td.string.replace(',', ''))

# 요구사항

- 통화명, 매매기준율, 사실때, 파실 때
- 통화코드까지 수집
  - 이런 데이터를 추출하기 위해서 사이트의 html구조를 꼼꼼하게 체크하기
  - 개발자 도구의 element파트에서 시뮬레이션 통해 특정한것이 정확한지 확인

- 반복적으로 등장하는 데이터 rows, list형태(tr, li태그들로 나열된 형태)
  - 이러한 데이터를 받게 될 땐, row데이터 단위로 수해야해야 누락없이 수집 가능
  - 그래야 [{},{},{}]구조로 형성 가능

In [ ]:
# (1) 통화정보를 가진 tr 44개를 수집하시오\
# 테이블 데이터는 행단위로 추출하는 것이 안전함.

In [ ]:
# div.tbl_area > table.tbl_exchange > tbody > tr
tmp = soup.select('div.tbl_area > table.tbl_exchange > tbody > tr')
len(tmp)

44

### 방법1

In [ ]:
res= []
for i in tmp:
  currency      = i.a.string.strip()
  code          = i.a.get('href')[-6:][:3]
  # -> 해당 키가 존재하지 않아도 오류는 안 난다: None
  # code          = i.a['href'][-6:][:3]
  trade_std_rate= i.select_one('.sale').text
  buy_curr      = i.select('td')[2].text
  sell_curr     = i.select('td')[3].text
  send_curr     = i.select('td')[4].text
  take_curr     = i.select('td')[5].text
  ex_to_dollar  = i.select('td')[6].text
  # print( code )

  dic = {'currency'     :currency,
         'code'         :code,
        'trade_std_rate':trade_std_rate,
         'buy_curr'     :buy_curr,
         'sell_curr'    :sell_curr,
         'send_curr'    :send_curr,
         'take_curr'    :take_curr,
         'ex_to_dollar':ex_to_dollar}
  res.append( dic )
  pass

### 방법2

- nth-of-child 사용

In [ ]:
res= []
for i in tmp:
  currency      = i.a.string.strip()
  code          = i.a.get('href')[-6:][:3]
  # -> 해당 키가 존재하지 않아도 오류는 안 난다: None
  trade_std_rate= i.select_one('.sale').text
  # nth-of-child -> nth-of-type형태로 바꿔서 사용하기(중요)
  buy_curr      = i.select_one('td:nth-of-type(3)').string
  sell_curr     = i.select_one('td:nth-of-type(4)').string
  send_curr     = i.select_one('td:nth-of-type(5)').string
  take_curr     = i.select_one('td:nth-of-type(6)').string
  ex_to_dollar  = i.select_one('td:nth-of-type(7)').string
  # print( code )

  dic = {'currency'     :currency,
         'code'         :code,
         'trade_std_rate':trade_std_rate,
         'buy_curr'     :buy_curr,
         'sell_curr'    :sell_curr,
         'send_curr'    :send_curr,
         'take_curr'    :take_curr,
         'ex_to_dollar':ex_to_dollar}
  res.append( dic )
  pass
# res

### json저장

- DB저장 가능
- DB: RDBMS, No-Sql

In [ ]:
import json

In [ ]:
# 저장
with open('fx.json', mode='w') as fp:
  json.dump( res, fp )

In [ ]:
# 로드
with open('fx.json', mode='r') as fp:
  print( json.load( fp ) )
  # [  { }, { }, { }, ... { } ]형태

[{'currency': '미국 USD', 'code': 'USD', 'trade_std_rate': '1,086.70', 'buy_curr': '1,105.71', 'sell_curr': '1,067.69', 'send_curr': '1,097.30', 'take_curr': '1,076.10', 'ex_to_dollar': '1.000'}, {'currency': '유럽연합 EUR', 'code': 'EUR', 'trade_std_rate': '1,313.66', 'buy_curr': '1,339.80', 'sell_curr': '1,287.52', 'send_curr': '1,326.79', 'take_curr': '1,300.53', 'ex_to_dollar': '1.209'}, {'currency': '일본 JPY (100엔)', 'code': 'JPY', 'trade_std_rate': '1,041.25', 'buy_curr': '1,059.47', 'sell_curr': '1,023.03', 'send_curr': '1,051.45', 'take_curr': '1,031.05', 'ex_to_dollar': '0.958'}, {'currency': '중국 CNY', 'code': 'CNY', 'trade_std_rate': '166.46', 'buy_curr': '174.78', 'sell_curr': '158.14', 'send_curr': '168.12', 'take_curr': '164.80', 'ex_to_dollar': '0.153'}, {'currency': '홍콩 HKD', 'code': 'HKD', 'trade_std_rate': '140.19', 'buy_curr': '142.95', 'sell_curr': '137.43', 'send_curr': '141.59', 'take_curr': '138.79', 'ex_to_dollar': '0.129'}, {'currency': '대만 TWD', 'code': 'TWD', 'trade_

In [ ]:
# 실습: 2020.12.10 13:22 하나은행 기준 고시회차 105회
# 이중 날짜+시간, 기관명, 회차 추출

# 웹스크래핑+html파싱(dom트리 생성)
url   = 'https://finance.naver.com/marketindex/?tabSel=exchange#tab_section'
res   =   urlopen( url )

soup  = BeautifulSoup( res,"html5lib" )

date      = soup.select_one('.exchange_info > span.date').text
standard  = soup.select_one('.exchange_info > span.standard').text.split()[0]
round     = soup.select_one('.exchange_info > span.round').text
num       = soup.select_one('.exchange_info > span > em').text

'2020.12.10 14:19'

- 회차 정보는 1개
- 1개 회자당 통화정보는 44개 존재
- 저장정보 방식 고민
  - 44개의 데이터에 회자정보를 다 붙여 저장? ( 1개의 테이블 사용, 중복 데이터 많아짐)
  - 회자 정보를 저장하는 테이블에 먼저 저장하고 여기서 나오는 인덱스 값을 이용해서 개별 44개 데이터에 추가하여 저장? 
  - -> join을 통해데이터 추출


# 바이너리 파일을 다운받아 수집

- 바이너리 파일: 압축파일
- 압축을 해제해야함

- 응용
  - 웹상에서 이미지 수집도 가능하다!
- 목적
  - http://yann.lecun.com/exdb/mnist/ 접속
  - 데이터 4개를 다운로드
    - train-images-idx3-ubyte.gz:  training set images (9912422 bytes)
    - train-labels-idx1-ubyte.gz:  training set labels (28881 bytes)
    - t10k-images-idx3-ubyte.gz:   test set images (1648877 bytes)
    - t10k-labels-idx1-ubyte.gz:   test set labels (4542 bytes)
  - 압축해제
  - 디코딩 수행
  - 원하는 데이터를 추출 및 저장

In [ ]:
# 1. 타겟 사이트  -> mnist(머신러닝시 1차적으로 사용하는 데이터 -> 마치 r의 iris)

In [ ]:
target_site   =  'http://yann.lecun.com/exdb/mnist/'
res           =   urlopen( target_site ) 
if res.getcode() == 200:
  print('정상적으로 잘 받았다')
else:
  print('사이트에 뭔가 문제가 있다 점검 필요')
# 웹스크래핑:soup까지 생성
soup    = BeautifulSoup( res, "html5lib" )

정상적으로 잘 받았다


In [ ]:
link = []
for i in soup.select('p > tt > a'):
  link.append(i.text)
link

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

In [ ]:
# 'http://yann.lecun.com/exdb/mnist/' 주소에 각 파일명 넣으면 바로 다운로드 가능
down_link =[]
for i in link:
  down_link.append( target_site + i )
down_link

['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
 'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
 'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
 'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

In [ ]:
len( soup.select('tt') )

49

In [ ]:
# 슬라이싱을 통해 상위 4개 tt만 출력해볼수도 있다.

# for i in soup.select('tt')[:4]:
#   print(i.a.string)

# 리스트 내포도 가능하다.
files  = [tt.a.string for tt in soup.select('tt')[:4] ]
files

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

In [ ]:
# 4. 파일 다운로드
# 원격지에 있는 파일과, 로컬상에 특정 위치(저장할 파일)까지 스크림을 열고 
# 읽어서 바로 기록    -> 운영체계가 하는 일을 프로그램으로

# 파일 저장에 필요한 팝업이 생략, 원하는 위치에 바로 저장( 스크래핑의 핵심 !!!!!!!!!! )
import os, os.path

In [ ]:
# 4-1. 저장할 파일 위치 선정
savedPath   = './data/mnist'
# 물리적으로 존재하지 않으면 폴더 생성
if not os.path.exists( savedPath ):
  # 디렉토리 생성
  os.makedirs( savedPath )
  print('다운로드 파일을 저장한 파일 폴더를 생성했다.')
else:
  pass

다운로드 파일을 저장한 파일 폴더를 생성했다.


In [ ]:
# 4-2. 파일 저장
# 진행률을 표시
import tqdm.notebook
# level2의 방법을 사용하여 해당 파일을 직접 요청한다.
import urllib.request as req

In [ ]:
#  tqdm.notebook.tqdm(연속 데이터 타입)   <- 진행률이 표기된다~~~~~~
for file in tqdm.notebook.tqdm(files):
  # print(file)
  # 파일 저장
  # 만약 해당 파일이 이미 존재한다면 생략한다.
  # 파일의 물리적 위치
  # file_path    = f'{savedPath}/{file}'  <- 이 방식도 가능하다.
  os.path.join( savedPath, file )       
  #  물리적 경로를 동적으로 만들고 싶다면 위의 방식 사용.
  break
  pass

In [ ]:
for file in tqdm.notebook.tqdm(files):
  path  = os.path.join( savedPath, file )       
  # 체킹: 이미 파일이 존재하면 수행 안 함
  if not os.path.exists( path ):
    # 파일 저장
    # 파일을 조금씩 읽어서 로컬 파일에 차곡차곡 기록한다. 
    # tcp/ip 기반의 http를 사용하므로 모든 패킷은 "순서대로" "손실없이" 파일에 저장됨  -> 신뢰를 기반
    req.urlretrieve( target_site+file, path)

# url만 알면 웹페이지에 있는 모든 파일을 불러올 수 있다!!
# colab에서 파일은 인스턴스. 끝나면 사라짐.

## 압축 데이터 해제

- 확장자: gz파일
  - gzip 모듈을 활용

In [ ]:
import gzip

In [ ]:
# 압축 푸는 것도 진행률 봐야하니까
for file in tqdm.notebook.tqdm(files):
  # 원본 파일 경로  -> 풀어서 어딘가에는 저장을 해야하기 때문
  path  = os.path.join( savedPath, file )
  # 대상 파일 경로:  .gz를 제거해서 파일명 생성
  rowFile_path  = os.path.join( savedPath, file[:-3] )
  # 압축파일 오픈   ->  "읽기"(용량이 크면 나눠서 읽어야, 작으면 한번에)  ->  "기록"
  with gzip.open( path, mode='rb') as fg:
    # 읽기 : 용량이 적어서 한번에 읽을 것. 압축이 해제된 내용으로 읽힌다.
    tmp   = fg.read()
    # 쓰기 : 파일 오픈
    with open(rowFile_path, 'wb' ) as f:
      # 기록
      f.write( tmp ) 
      # 압축 내용이 풀려서 저장
  # print(rowFile_path)
  

## 데이터 디코딩

- mnist에서 손글씨 인식에 대한 데이터를 제공
  - 훈련용 데이터 6만개, 테스트용 데이터가 1만개
  - 압축해서 제공 -> 6만개를 압축한게 아니라 하나의 파일로 만들어서 압축한 후 제공
  - 1개의 파일을 만들려면, 나름대로 파일 구조가 필요
  - 이런 구조에 맞춰져서 파일이 만들어지면 인코딩(자체 포멧)
  - 인코딩된 파일을 6만개의 개별 파일로 분리하려면 구조(포멧)를 알고 디코딩을 수행해야함

- 포멧
  - 포멧을 이해해야 전략을 수립할 수 있다.
  - 8 bit -> 1 bit
  - 1 bit -> 0과 1을 값으로 가질 수 있다.
  - 메모리를 비트 단위로 구성
  - TRAINING SET LABEL FILE
    - 0번 위치부터 출발, 32 bit(4 byte) int,  magic number(표식=2049)
    - 4번위치, 4 byte int :  item의 개수( LABEL = 정답 = 종속변수 = Class , 손글씨 0,1,2,3, ...) 
    - 5번부터 끝까지 단위는 1 byte 단위: 1 unsigned byte( 부호가 없는 양수 ), 값은 0~9 사이의 어떤값.
    - 데이터의 전체 크기는 
      - train : (4  + 4  + 1*60,000) byte
      - test  : (4  + 4  + 1*10,000) byte
      

- TRAINING SET IMAGE FILE
  - 0번 위치부터 출발, 32 bit(4 byte) int, magic number(표식=2051)
  - 4번위치, 4 byte int : 이미지의 개수( 독립변수 = Feature, 특성, 손글씨 이미지의 개수 )
  - 8번 위치  : 4 byte int, 이미지의 rows
  - 12번 위치 : 4 byte int, 이미지의 cols
  - 16번부터 끝까지 : 1 unsigned byte( 0 ~ 255의 컬러값을 표현 ) , pixel
  - 데이터의 전체 크기
    - train : 4 + 4 + 4 + 4 + (rowsxcols)(이게 이미지 하나) x 60000(이미지 개수)
    - test  : 4 + 4 + 4 + 4 + (rowsxcols)(이게 이미지 하나) x 10000(이미지 개수)
  - 이미지는 사각형의 모습을 가짐( 2d 이미지 )
    - 가로, 세로의 정보 ( 세로: rows, 가로: cols )

## 데이터 추출해서 파일 저장

- pgm(샘플저장), csv에 저장(픽셀정보)|

In [ ]:
# 바이너리 데이터 (정수부분) 읽을 때 사용하는 모듈
import struct

- os별, cpu 벤더별(인텍, AMD, Apple) 정수값을 기록하는 방식이 다름(2, 4, 8, ...)
  - 에디안 방식이 빅에디안, 리틀에디안으로 나뉜다.
    - ex) 0x12345678 : 16진법, 0xFF = 255
      
      


- 빅에디안으로 메모리에 기록을 한다면?
- high endian

|주소|0x001|0x002|0x003|0x004|
|--|--|--|--|--|
|값|0x12|0x34|10x56|0x78|

- 리틀에디안으로 메모리에 기록을 한다면?
- low-endian

|주소|0x001|0x002|0x003|0x004|
|--|--|--|--|--|
|값|0x78|0x56|0x34|0x12|

- 본 파일은 high endian으로 기록 됨
- struct는 
  - '>' : 빅에디안
  - '<' : 리틀 에디안
  - 'I' : 4 byte 읽는다

In [ ]:
# /content/data/mnist/train-labels-idx1-ubyte
# 이 파일을 디코딩해서 csv파일에 저장하기
# 비정형 데이터(바이너리 데이터, 이미지) -> 반정형 데이터로 구성
file_str  = '/content/data/mnist/train-labels-idx1-ubyte'
with open(file_str, mode='rb') as f:
  # 바이너리 파일의 헤더 정보 4+4 바이트를 먼저 읽는다.
  # f.read( ) : end of file까지 읽는다. 그리고 바이너리는 못 읽음

  # 1.매직넘버 읽기: 빅에디안 4바이트만 읽는다.
  magic_number  =   struct.unpack( '>I', f.read(4) )
  print(magic_number)

  # 파일을 읽으면 읽어야 하는 시작 위치가 읽은 값의 크기만큼 이동 : 4바이트에서 커서
  # 2. 아이템의 개수(레이블 총개수)
  label_count  =   struct.unpack( '>I', f.read(4) )
  print(label_count[0])
  pass
  # unpack()은 튜플로 리턴 -> 바이트 포멧단위로 여러개를 리턴할 수 있다.

(2049,)
60000


In [ ]:
file_str  = '/content/data/mnist/train-labels-idx1-ubyte'
with open(file_str, mode='rb') as f:
  # 4바이트 2개씩 읽는다
  # 에디안 방식으로 읽어야할 항목들을 나열하여 한번에 다 처리
  magic_number, label_count  =   struct.unpack( '>II', f.read(4+4) )
label_count

60000

In [ ]:
# 레이블을 각각 읽어서 csv에다가 저장
# 1) 읽기 2) 쓰기
with open(file_str, mode='rb') as f:
  magic_number, label_count  =   struct.unpack( '>II', f.read(4+4) )
  # 총 1 byte를 label_count만큼 읽어야 한다.
  for idx in range(label_count): # 반복수: 0부터 label_count-1까지
    # 1 byte를 60000번 읽는다.( 부호 없음: unsigned byte ) : 0~2^(8-1): 0 ~ 255
    # 실제적으로 세팅된 값은 0,1,2~9 : 손글씨이미지 0부터 9까지 기록
    (label,)  = struct.unpack( 'B', f.read(1) )
    print(label)
    break

5


In [ ]:
os.path.join( savedPath , 'a.csv' )

'./data/mnist/a.csv'

In [ ]:
file_str

'/content/data/mnist/train-labels-idx1-ubyte'

In [ ]:
# 2) 쓰기
with open(file_str, mode='rb') as f:
  # (1) csv 오픈
  csv_path  = os.path.join( savedPath , 'train-labels-idx1.csv' )
  with open(csv_path, 'w' ) as fp:      
    magic_number, label_count  =   struct.unpack( '>II', f.read(4+4) )
    if magic_number == 2049: #  이 파일은 레이블 파일
      for idx in range(label_count): 
        (label,)  = struct.unpack( 'B', f.read(1) )
        # (2) 줄바꿈 기호 추가하여 저장 -> csv는 최종적으로 60000줄
        fp.write( str(label)+ '\n')

In [ ]:
# 이미지 저장
file_str  = '/content/data/mnist/train-images-idx3-ubyte'
with open(file_str, mode='rb') as f:
  _, imgCnt, rows, cols  =   struct.unpack( '>IIII', f.read(4+4+4+4) )

2051 60000 28 28


- 이미지의 크기는
- 4+4+4+4 + (28 * 28)* 60000 = 47,040,016 byte

In [ ]:
' '.join('12345')
# 사이사이 넣기

'1 2 3 4 5'

In [ ]:
with open(file_str, mode='rb') as f:
  _, imgCnt, rows, cols  =   struct.unpack( '>IIII', f.read(4+4+4+4) )
  # 픽셀 크기 지정
  pixels = rows * cols # 하나당 784 바이트
  for idx in range( imgCnt ):
    # 파일 1개를 디코딩 해서 확인(정상 파일인지)
    header  = f'P2 {rows} {cols} 255\n' # 컬러의 총 수 255
    data    = f.read(pixels) 
    # data 프린트해보면 b표시 : byte로 표시된 문자열
    src     = header + ' '.join(list( map( str, data ) ))  + '\n'
    # 파일저장
    with open( 'test.pgm', 'w', encoding='utf-8') as fp:
      fp.write( src )
    break

In [ ]:
# 구성원들 하나한 일일이 접근해서 뭔가 작업을 수행하고 싶다면
# map( 함수, 연속데이터 타입) 
'''
  1) b"\x00\x00\x
  2) ['2','3','4']
  3) '2 3 4
'''
' '.join(list( map( str, data ) ))

'0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 18 18 18 126 136 175 26 166 255 247 127 0 0 0 0 0 0 0 0 0 0 0 0 30 36 94 154 170 253 253 253 253 253 225 172 253 242 195 64 0 0 0 0 0 0 0 0 0 0 0 49 238 253 253 253 253 253 253 253 253 251 93 82 82 56 39 0 0 0 0 0 0 0 0 0 0 0 0 18 219 253 253 253 253 253 198 182 247 241 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 80 156 107 253 253 205 11 0 43 154 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 14 1 154 253 90 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 139 253 190 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 11 190 253 70 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 35 241 225 160 108 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 81 240 253 253 119 25 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
','.join(list( map( str, data ) ))
# csv에 저장할 땐 이렇게

'0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,18,18,18,126,136,175,26,166,255,247,127,0,0,0,0,0,0,0,0,0,0,0,0,30,36,94,154,170,253,253,253,253,253,225,172,253,242,195,64,0,0,0,0,0,0,0,0,0,0,0,49,238,253,253,253,253,253,253,253,253,251,93,82,82,56,39,0,0,0,0,0,0,0,0,0,0,0,0,18,219,253,253,253,253,253,198,182,247,241,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,156,107,253,253,205,11,0,43,154,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,1,154,253,90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,139,253,190,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,190,253,70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35,241,225,160,108,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,81,240,253,253,119,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
with open(file_str, mode='rb') as f:
  csv_path  = os.path.join( savedPath , 'train-images-idx3-ubyte.csv' )
  with open(csv_path, 'w' ) as fp:      
    _, imgCnt, rows, cols  =   struct.unpack( '>IIII', f.read(4+4+4+4) )
    pixels = rows * cols 
    for idx in range( imgCnt ): # 한 줄씩 기록
      data    = f.read(pixels) 
      tmp = ','.join(list( map( str, data ) )) + '\n'
      # 구분자는 , 한 개의 데이터(이미지)의 끝은 줄바꿈 기호 추가
      fp.write( tmp )

In [ ]:
savedPath

'./data/mnist'

In [ ]:
# http://paulcuth.me.uk/netpbm-viewer/ 에서 저장된 pgm파일 확인